# BGSE Text Mining Homework 2
### Euan Dowers, Veronika Kyuchukova, and Laura Roman

#### Exercise 1

The objective of this exercise is to implement uncollapsed gibbs sampling for fitting an LDA model to state of the union speeches from 1945 onwards, with documents being defined at the paragraph level. 

First, we need to read in and process the data, as in the first homework set:

In [1]:
import numpy as np
import pandas as pd
import nltk
import os
import sys
import scipy.sparse as ssp
import time
import matplotlib
import tqdm
from tqdm import tqdm
from numpy.random import dirichlet, multinomial
from collections import Counter
from utils import data_processing, get_vocab, make_count
%matplotlib inline

In [2]:
data = pd.read_table("HW1/speech_data_extend.txt",encoding="utf-8")
data_post1945 = data.loc[data.year >= 1945]
%time stemmed, processed_data = data_processing(data_post1945)

CPU times: user 8.57 s, sys: 36 ms, total: 8.6 s
Wall time: 8.83 s


Now we will create a function that implements uncollapsed gibbs sampling on our processed data.
This essentially works by repeatedly sampling from the posterior distributions of $Z$, $\Theta$, and $\beta$ and updating values using the most recent sample. 

In [3]:
def Gibbs_sampling_LDA(stemmed, K, alpha = None, eta = None, m=3, n_samples = 200, burnin = 500, perplexity = False):
    '''
    Gibbs sampler for LDA model
    '''

    def Z_sample(Beta, Theta, Theta_x_Beta):
        denom = Theta_x_Beta
        Z = [0] * len(stemmed)
        for i in range(len(stemmed)):
            sel = [idx[word] for word in stemmed[i]]
            prod = Beta[:,sel].T * Theta[i,:].reshape((1,K))
            probs = prod / denom[i,sel].reshape(len(stemmed[i]),1)
            Z[i] = [np.argmax(multinomial(n=1, pvals = z, size = 1)[0]) for z in probs]
        return Z

    def Beta_sample(eta, Z):
        z_s = [z for sublist in Z for z in sublist ]
        M = np.zeros(shape=(K,V))
        for k in range(K):
            words = [s[i] for i in range(len(z_s)) if z_s[i] == k]
            counts = Counter(words)
            for word in set(words):
                M[k,idx[word]] = counts[word]
        Beta = [dirichlet(alpha = eta + M[i],size = 1)[0] for i in range(K)]
        return np.array(Beta)

    def Theta_sample(alpha, Z):
        N   = np.zeros(shape=(D,K))
        for i in range(D):
            counts   = Counter(Z[i])
            for j in set(counts.keys()):
                N[i,j]  = counts[j]
        Theta = [dirichlet(alpha = alpha + N[i],size = 1)[0] for i in range(D)]
        return np.array(Theta)

    def perplexity(Theta_x_Beta, count_matrix):
        '''
        Calculate perplexity for given sample
        '''
        ltb     = np.log(Theta_x_Beta)
        num     = np.sum(count_matrix.multiply(ltb))
        denom   = len(s)
        return np.exp(-num/denom)

    # Get params needed for passing to sampling functions
    s       = [i for sublist in stemmed for i in sublist ]
    vocab   = get_vocab(stemmed)
    D       = len(stemmed)
    V       = len(vocab)
    idx     = dict(zip(vocab,range(len(vocab))))
    count_matrix = make_count(stemmed, idx)
    perp   = []

    # Initialise params
    if eta == None:
        eta = 200/V
    if alpha == None:
        alpha = 50/K

    Theta   = dirichlet(alpha = [alpha]*K, size = D)
    Beta    = dirichlet(alpha = [eta]*V, size = K)
    Theta_x_Beta = Theta.dot(Beta)
    Z       = Z_sample(Beta, Theta, Theta_x_Beta)
    labels  = np.zeros((n_samples, len(s)))

    # SAMPLING
    print('TIME:', time.strftime("%H:%M:%S", time.gmtime()))
    for i in tqdm(range(burnin)):
        Z       = Z_sample(Beta, Theta, Theta_x_Beta)
        Beta    = Beta_sample(eta, Z)
        Theta   = Theta_sample(alpha, Z)
        Theta_x_Beta = Theta.dot(Beta)
        if i%20 == 0:
            if perplexity:
                perp.append(perplexity(Theta_x_Beta, count_matrix))
            #print('Burnin iteration {}'.format(i))

    print('TIME:', time.strftime("%H:%M:%S", time.gmtime()))
    for i in tqdm(range(m*n_samples)):
        Z       = Z_sample(Beta, Theta, Theta_x_Beta)
        Beta    = Beta_sample(eta, Z)
        Theta   = Theta_sample(alpha, Z)
        Theta_x_Beta = Theta.dot(Beta)

        # Add every m-th sample to output
        if i%m == 0:
            Z_s = [i for sublist in Z for i in sublist ]
            j = np.int(i/m)
            labels[j, :] = Z_s
        if i%20 == 0:
            if perplexity:
                perp.append(perplexity(Theta_x_Beta, count_matrix))
            #print( "Iteration {}".format(i))

    return (labels, perp)

In [4]:
LDA_labels, perp = Gibbs_sampling_LDA(stemmed,
                                      K = 10,
                                      n_samples = 100,
                                      m = 5,
                                      perplexity=True,
                                      burnin = 1000)

  0%|          | 0/1000 [00:00<?, ?it/s]

TIME: 06:39:10


  0%|          | 0/500 [00:00<?, ?it/s]

TIME: 07:30:21


100%|██████████| 500/500 [25:20<00:00,  3.50s/it]


As you can see this sampling function takes around 1h10m to complete 1500 iterations. The output is a list of perplexities (from every 20th sample) and a numpy array containing a label assignment for each word in the corpus, for each sampling iteration after the initial burnin period. Since this takes so long we have saved these outputs and will load them from disk for any further analysis. 

In [5]:
LDA_labels.tofile('LDA_labels.npy')
pd.DataFrame(perp).to_csv('perplexity.csv')

Now make a function that, given a draw from the gibbs sampling, returns a document topic matrix (for use in the next exercise).

In [6]:
doc_label = [[i]*len(stemmed[i]) for i in range(len(stemmed))]
doc_label = [i for sublist in doc_label for i in sublist]

def dt_matrix(labels, doc_label):
    dt = np.zeros((len(set(doc_label)), K))
    label_dict = dict(zip(range(labels.shape[0]), labels.astype(int)))
    doc_dict = dict(zip(range(len(doc_label)), doc_label))
    for i in range(len(doc_label)):
        dt[doc_dict[i], label_dict[i]] += 1
    return dt

Now use this to come up with some kind of estimate of the predictive distribution of theta

In [28]:
alpha = 50/10
K = 10
dt_avg = np.zeros((len(stemmed), 10))
for i in np.arange(0,100, 10 ):
    dt_avg += dt_matrix(LDA_labels[i], doc_label)
dt_avg /= 10

nmz = dt_avg
nm = np.array([len(doc) for doc in stemmed]).reshape(10252,1)

theta_unc = (dt_avg + alpha) / (nm + 10 * alpha)

theta_unc = pd.DataFrame(theta_uncollapsed)
theta_unc.to_csv('theta_uncollapsed.csv')

Here we calculate document term matrices for a number of draws and then average them to estimate the predictive distribution of $\Theta$

#### Exercise 2

The objective of this exercise is to run the collapsed gibbs sampling version for fitting an LDA model to the same data, hyperparameters and K. 


We will work with a Python3 adapted version of the collapsed Gibbs sampler that can be found in https: //github.com/sekhansen/text-mining-tutorial 

a) Plot the perplexity across sampling iterations. Which algorithm appears to burn in faster?

In [ ]:
import topicmodels

# 2.1 

ldaobj = topicmodels.LDA.LDAGibbs(stemmed, 10)


ldaobj.sample(0, 20, 75)


perp_2 = ldaobj.perplexity()

perp_1 = perp


plt.plot(perp_1,   lw = 1., label = 'uncollapsed')
plt.plot(perp_2, lw = 1., label = 'collapsed')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.ylabel('perplexity')
plt.savefig('perplexities_collapsed.png', bbox_inches='tight')


# 2.2

ldaobj = topicmodels.LDA.LDAGibbs(stemmed, 10)
ldaobj.sample(1000, 5, 100)


k = ldaobj.K
alpha =  ldaobj.alpha

docterms = ldaobj.dt_avg()
nm = np.array([len(doc) for doc in stemmed])
nm = nm.reshape((10252,1))
nmz = nm*docterms
nmz

theta_coll = (nmz+alpha)/(nm+k*alpha)

#compare theta for different topics and map collapsed-uncollapsed
theta.sum(axis=0)

#from uncollapsed 
theta_uncoll = theta_uncollapsed

theta_uncoll.drop( 'Unnamed: 0',axis=1,inplace=True)
theta_uncoll.sum(axis=0)

theta_uncoll= np.array(theta_uncoll)

# idea 2: compar mean and sd

m1 = theta.mean(axis=0)
st1= theta.std(axis=0)

m2 = theta_uncoll.mean(axis=0)
st2 = theta_uncoll.std(axis=0)
data_compare = pd.DataFrame([m1,m2,st1,st2])

# idea 3: compare the histograms/plots

t1 = theta[:,0]
t2 = theta[:,1]
t3 = theta[:,2]
t4 = theta[:,3]
t5 = theta[:,4]
t6 = theta[:,5]
t7 = theta[:,6]
t8 = theta[:,7]
t9 = theta[:,8]
t10 = theta[:,9]

ut1 = theta_uncoll[:,0]
ut2 = theta_uncoll[:,1]
ut3 = theta_uncoll[:,2]
ut4 = theta_uncoll[:,3]
ut5 = theta_uncoll[:,4]
ut6 = theta_uncoll[:,5]
ut7 = theta_uncoll[:,6]
ut8 = theta_uncoll[:,7]
ut9 = theta_uncoll[:,8]
ut10 = theta_uncoll[:,9]


#T1-UT2
#T2-UT5


bins = np.linspace(0.03, 0.225, 70)

pyplot.hist(t8, bins, alpha=0.5, label='collapsed')
pyplot.hist(ut1, bins, alpha=0.5, label='uncoll, t1')
pyplot.hist(ut2, bins, alpha=0.5, label='uncoll, t2')
pyplot.hist(ut3, bins, alpha=0.5, label='uncoll, t3')
pyplot.hist(ut4, bins, alpha=0.5, label='uncoll, t4')
pyplot.hist(ut5, bins, alpha=0.5, label='uncoll, t5')
pyplot.hist(ut6, bins, alpha=0.5, label='uncoll, t6')
pyplot.hist(ut7, bins, alpha=0.5, label='uncoll, t7')
pyplot.hist(ut8, bins, alpha=0.5, label='uncoll, t8')
pyplot.hist(ut9, bins, alpha=0.5, label='uncoll, t9')
pyplot.legend(loc='upper right')
pyplot.show()
